<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing_ubs_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-26
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Segmenting text by bank name and analyst name
    - Pairing question and answer by GPT 4
    - Cleaning Texts
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-26\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Segmenting text by bank name and analyst name\n    - Pairing question and answer by GPT 4\n    - Cleaning Texts\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [249]:
!pip install python-dotenv

In [250]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive
import openai
import json
from dotenv import load_dotenv
import os
from google.colab import userdata
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [251]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


In [252]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

## **Structuring Data Frame**

In [253]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [254]:
#Defining qa_data
qa_data = pd.read_csv("/content/ubs_qna_section.csv")
qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file
0,Unknown,NaN,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
2,Chris Hallam,Goldman Sachs,"Very clear. Thanks. Kian Abouhossein, JPMorgan...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
3,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
4,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf


In [255]:
"""
Defining lists of Exective names from UBS
"""
exective_name = ["Sergio P. Ermotti","Sarah Youngwood	"]
exect_name = r'\b(Sergio P\. Ermotti|Sarah Youngwood)\b'


In [256]:
"""
This code searches for Analyst / Exective name and their bank name in the 'utterance' column of a DataFrame and stores any matches in a new 'dummy' column. If no matches are found, the 'dummy' column remains None for that row.

"""
pattern = r'\b[A-Z][a-z]+ [A-Z][a-z]+, [A-Z][A-Za-z]+'
qa_data["dummy"]=None
for i in range(len(qa_data)):
  matches = re.findall(pattern, str(qa_data['utterance'][i]))
  if matches:
    qa_data.at[i, 'dummy'] = matches
  else:
    continue

qa_data["ex_dummy"]=None
for i in range(len(qa_data)):
  matches1 = re.findall(exect_name, str(qa_data["utterance"][i]))
  if matches1:
    qa_data.at[i,"ex_dummy"]=matches1
    qa_data.loc[i,"ex_dummy"].append("UBS")
  else:
    continue


In [257]:
qa_data.head(50)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
0,Unknown,NaN,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
2,Chris Hallam,Goldman Sachs,"Very clear. Thanks. Kian Abouhossein, JPMorgan...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
3,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
4,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
5,Sarah Youngwood,NaN,That's right. The initial PPA comes into the C...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
6,Sergio P. Ermotti,NaN,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
7,Alastair Ryan,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
8,Sergio P. Ermotti,NaN,"Thank you, Ryan. It is good to be back to inte...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
9,Alastair Ryan,Bank of America,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None


In [258]:
"""
This code calculates the number of rows with non-null values in the 'dummy' column and the total number of rows in the DataFrame.
"""
additional_row = len(qa_data) - qa_data["dummy"].isnull().sum()
print("The number of new rows: ", additional_row)
total = additional_row + len(qa_data)
print("Total number of rows: ", total)

The number of new rows:  56
Total number of rows:  365


In [259]:
"""
This code searches for Analyst/Exective name and their bank name in the 'utterance' column of a DataFrame, updates the 'speaker' column for the first row with matches, and splits the 'utterance' and inserts a new row for subsequent matches. If no matches are found, the loop continues to the next row.
"""
# I have to loop the parts 1[0] so that it catches the second bank as well.


for  i in qa_data.index:
  matches = re.findall(pattern, str(qa_data['utterance'][i]))
  matches1 = re.findall(exect_name, str(qa_data['utterance'][i]))
  if matches and i==0:
    print(matches)
    qa_data.at[i, 'speaker'] = matches
    qa_data.at[i,"job_title"]= matches
  elif matches:
    if len(qa_data["dummy"][i]) <2:
      new_index=i+0.5
      parts1 = [part.strip() for part in qa_data['utterance'][i].split(matches[0])]
      qa_data.at[i, 'utterance'] = parts1[0]
      qa_data.loc[new_index] = {"speaker":matches,"job_title":matches,"utterance":parts1[1], "call_date":qa_data["call_date"][i], "financial_quarter":qa_data["financial_quarter"][i],"source_file":qa_data["source_file"][i], "dummy":None}
    else:
      print(matches[0])
      print(matches[1])
      parts1 = [part.strip() for part in qa_data['utterance'][i].split(matches[0])]
      parts2 = [part.strip() for part in qa_data['utterance'][i].split(matches[1])]
      parts3 = [part.strip() for part in parts1[1].split(matches[1])]
      qa_data.at[i, 'utterance'] = parts1[0]
      qa_data.loc[i+0.2] = {"speaker":matches[0],"job_title":matches[0],"utterance":parts3[0], "call_date":qa_data["call_date"][i], "financial_quarter":qa_data["financial_quarter"][i],"source_file":qa_data["source_file"][i], "dummy":None}
      qa_data.loc[i+0.4] = {"speaker":matches[1],"job_title":matches[1],"utterance":parts2[1], "call_date":qa_data["call_date"][i], "financial_quarter":qa_data["financial_quarter"][i],"source_file":qa_data["source_file"][i], "dummy":None}
      print(parts1[0])
      print("BREAK123")
      print(parts3[0])
      print("BREAK123")
      print(parts2[1])
      print("BREAK456")

      #qa_data.at[i, 'utterance'] = parts1[0]
  elif matches1:
    new_index=i+0.5
    parts2 = [part.strip() for part in qa_data['utterance'][i].split(matches1[0])]
    qa_data.at[i, 'utterance'] = parts2[0]
    qa_data.loc[new_index] = {"speaker":qa_data["ex_dummy"][i],"job_title":"UBS","utterance":parts2[1], "call_date":qa_data["call_date"][i], "financial_quarter":qa_data["financial_quarter"][i],"source_file":qa_data["source_file"][i], "dummy":None}
  else:
    continue


['Chis Hallam, Goldman']
Anke Reingen, RBC
Amit Goel, Barclays
So on the treasury share, what happened is we actually repurchased some of the shares and they are available. They could have been cancelled and what we're doing is we are using them for the purpose of this acquisition. So, we are not issuing any new shares. And so in terms of commercial real estate, we have a very conservative portfolio that is – that has exposure to Switzerland in particular. But – and all of that is according to what you see from us, it was the high collateralization that you would expect. And when we give you all of our levels of reserves, we certainly reflect on the very high quality of those portfolio.
BREAK123
Okay. Thank you.
BREAK123
Hi. Thank you and welcome back, Sergio, too from me. So, two questions. The first, again, just on the transaction and the announcement on – on day one, I just wanted to check in terms of the cost savings and the timeline for EPS accretion whether or not your conviction

In [260]:
#Reset index due to new rows
qa_data=qa_data.sort_index().reset_index(drop=True)
qa_data.head(50)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
0,"[Chis Hallam, Goldman]","[Chis Hallam, Goldman]","Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
2,Chris Hallam,Goldman Sachs,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
3,"[Kian Abouhossein, JPMorgan]","[Kian Abouhossein, JPMorgan]",Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
4,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
5,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
6,"[Kian Abouhossein, JPMorgan]","[Kian Abouhossein, JPMorgan]",Sorry if I may. The 74% also includes - but do...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
7,Sarah Youngwood,NaN,That's right. The initial PPA comes into the C...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
8,"[Kian Abouhossein, JPMorgan]","[Kian Abouhossein, JPMorgan]",Ok.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
9,Sergio P. Ermotti,NaN,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None


In [261]:
"""
This code checks if the 'speaker' column contains a list, splits the first element of the list at the comma, and updates the 'speaker' and 'job_title' columns accordingly. If the 'speaker' is not a list, the loop continues to the next row.

"""
for i in range(len(qa_data)):
  if isinstance(qa_data['job_title'][i], list):
    parts = [part.strip() for part in qa_data['speaker'][i][0].split(',')]
    qa_data.at[i, 'speaker'] = parts[0]
    qa_data.at[i, 'job_title'] = parts[1]

  else:
    continue



In [262]:
qa_data.head(60)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
2,Chris Hallam,Goldman Sachs,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
4,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
5,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
6,Kian Abouhossein,JPMorgan,Sorry if I may. The 74% also includes - but do...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
7,Sarah Youngwood,NaN,That's right. The initial PPA comes into the C...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
8,Kian Abouhossein,JPMorgan,Ok.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
9,Sergio P. Ermotti,NaN,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None


In [263]:
"""
#Manual adjustments!!
"""

#row 56
text = str(qa_data["speaker"][56]) +str(qa_data["job_title"][56]) +str(qa_data["utterance"][56])
qa_data.at[55,"utterance"]=text
qa_data=qa_data.drop(index=56)
qa_data.reset_index(drop=True, inplace=True)
#qa_data=qa_data.drop(columns="dummy")

#row72
text = str(qa_data["speaker"][72]) +str(qa_data["job_title"][72]) +str(qa_data["utterance"][72])
qa_data.at[71,"utterance"]=text
qa_data=qa_data.drop(index=72)
qa_data.reset_index(drop=True, inplace=True)
#qa_data=qa_data.drop(columns="dummy")


#job_title for Goldman
qa_data.at[2,"job_title"]="Goldman"

#Remove last row
qa_data=qa_data.drop(index=len(qa_data)-1)
qa_data.reset_index(drop=True, inplace=True)

In [264]:
qa_data.head(60)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
2,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
4,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
5,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
6,Kian Abouhossein,JPMorgan,Sorry if I may. The 74% also includes - but do...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
7,Sarah Youngwood,NaN,That's right. The initial PPA comes into the C...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
8,Kian Abouhossein,JPMorgan,Ok.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
9,Sergio P. Ermotti,NaN,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None


In [265]:
qa_data["category"]=None
current_bank=qa_data["job_title"][0]
for i in range(len(qa_data)):
  if current_bank == qa_data["job_title"][i]:
    qa_data.at[i,"category"]=current_bank
  elif current_bank!=qa_data["job_title"][i] and qa_data["job_title"][i] is np.NaN:
    qa_data.at[i,"category"]=current_bank
  else:
    current_bank=qa_data["job_title"][i]
    qa_data.at[i,"category"]=current_bank


In [266]:
#add ubs in nan in job title
for i in range(len(qa_data)):
  if qa_data["job_title"][i] is np.NaN:
    qa_data.at[i,"job_title"]="UBS"

In [267]:
#Remove Caustionary statement
statement = r'\b(Cautionary Statement Regarding)\b'
for i in range(len(qa_data)):
  matches = re.findall(statement, str(qa_data["utterance"][i]))
  if matches:
    parts = [part.strip() for part in qa_data['utterance'][i].split(matches[0])]
    qa_data.at[i, 'utterance'] = parts[0]
    print(parts[0])
  else:
    continue

– closing remarks Okay. Looks like there are no more questions. Thank you for being with us this morning. As I said, despite the very uncertain market conditions, we delivered solid underlying results for the quarter. We have been maintaining our capital and liquidity positions, and we were also able to get very close to resolve a legacy matter that has been with us for 15 years. We discussed also the unique opportunity that the acquisition of Credit Suisse presents not only for our shareholders, but I believe also for all the rest of our stakeholders, employees, clients, society at large. And of course, this is not going to be a straight line journey. You know, it's going to be a challenging one, is a complex one. But I do believe that, you know, the benefits outweigh by far, all those factors. So, I know that you are looking for more details. We are also you are looking for more details. And we will really do our best to keep the communication channels open over the next few weeks an

In [268]:
qa_data.isnull().sum()

,0
speaker,0
job_title,0
utterance,0
call_date,0
financial_quarter,0
source_file,0
dummy,310
ex_dummy,364
category,0


In [269]:
qa_data[130:150]

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category
130,Todd Tuckner,UBS,Yeah. That was our understanding from Credit S...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Mediobanca
131,Adam Terelak,Mediobanca,Okay. Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Mediobanca
132,Andrew Coombs,Citigroup,Good morning it’s Andrew Coombs from Citi and ...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Citigroup
133,Todd Tuckner,UBS,"So, Andrew, in terms of – I’ll take the second...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Citigroup
134,Andrew Coombs,UBS,Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Citigroup
135,Vishal Shah,Morgan Stanley,Hi. Thank you so much for your questions. My f...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
136,Todd Tuckner,UBS,"Hey, Vishal. I mean I think on this on that se...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
137,Vishal Shah,Morgan Stanley,Okay. Thank you so much.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
138,Sergio P. Ermotti,UBS,Okay. The last answer and questions and I'm su...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
139,Giulia Miotto,Morgan Stanley,Hi. Good morning. Thank you for taking my ques...,14 August 2024,2Q24,2q24-earnings-call-remarks.pdf,None,None,Morgan Stanley


In [270]:
qa_data.to_csv("ubs_data.csv", index=False)

In [271]:
"""
qa_data["Text"]=None
text=""
for i in range(len(qa_data)):
  if i==0:
    text += qa_data["utterance"][i]
  elif qa_data["category"][i] != qa_data["category"][i-1]:
    qa_data.at[i-1,"Text"]=text
    text=""
    text += str(qa_data["speaker"][i]) + ", "
    text += str(qa_data["job_title"][i]) +" "
    text += str(qa_data["utterance"][i])
  else:
    text += str(qa_data["speaker"][i]) + ", "
    text += str(qa_data["job_title"][i]) + " "
    text += str(qa_data["utterance"][i])
"""

'\nqa_data["Text"]=None\ntext=""\nfor i in range(len(qa_data)):\n  if i==0:\n    text += qa_data["utterance"][i]\n  elif qa_data["category"][i] != qa_data["category"][i-1]:\n    qa_data.at[i-1,"Text"]=text\n    text=""\n    text += str(qa_data["speaker"][i]) + ", "\n    text += str(qa_data["job_title"][i]) +" "\n    text += str(qa_data["utterance"][i])\n  else:\n    text += str(qa_data["speaker"][i]) + ", "\n    text += str(qa_data["job_title"][i]) + " "\n    text += str(qa_data["utterance"][i])\n'

In [272]:
qa_data.head(50)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None,Goldman
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None,Goldman
2,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None,Goldman
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN,JPMorgan
4,Sergio P. Ermotti,UBS,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None,JPMorgan
5,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None,JPMorgan
6,Kian Abouhossein,JPMorgan,Sorry if I may. The 74% also includes - but do...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN,JPMorgan
7,Sarah Youngwood,UBS,That's right. The initial PPA comes into the C...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None,JPMorgan
8,Kian Abouhossein,JPMorgan,Ok.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN,JPMorgan
9,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None,JPMorgan


In [273]:
#qa_data.drop(columns=["job_title","utterance"],inplace=True)
#filtered_data = qa_data.dropna(subset="Text")

In [274]:
#filtered_data.head()

In [275]:
#filtered_data.reset_index(drop=True, inplace=True)

In [276]:
#filtered_data.head(20)

In [277]:
#filtered_data.to_csv("/content/ubs_qna_sectionver1.csv", index=False)

# **Export the output as a csv file**

UBS QA section

In [278]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/ubs_qa_df_preprocessed.csv"
processed_df.to_csv(preprocessed_qa_csv_path3, index=False)

NameError: name 'processed_df' is not defined

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)